In [1]:
from transformers import ViTModel
from transformers import ViTFeatureExtractor
from PIL import Image

model_save_path = 'model'
feature_extractor_save_path = 'feature_extractor'
model = ViTModel.from_pretrained(model_save_path)
image = Image.open('data/test/asparagus_test.jpg')
feature_extractor = ViTFeatureExtractor.from_pretrained(feature_extractor_save_path)
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
embeddings = outputs.last_hidden_state[:, 0, :]
print("Embedding vector size:", embeddings.size())
print("Embedding vector:", embeddings)

d:\workdir\python\venv\similarPic\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of ViTModel were not initialized from the model checkpoint at model and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\workdir\python\venv\similarPic\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Embedding vector size: torch.Size([1, 768])
Embedding vector: tensor([[-3.5563e-01, -1.1099e-01, -9.0854e-01,  7.5112e-01,  1.8149e+00,
         -8.1925e-02,  1.3351e+00,  9.4039e-02,  1.1419e+00, -6.5817e-02,
          2.6672e+00, -1.1579e+00,  2.0463e-01,  4.5644e-01,  6.7676e-01,
          1.3361e+00, -5.9456e-01, -1.2712e+00,  2.0390e-03, -7.0385e-01,
         -3.9839e-02,  7.2104e-01,  6.9824e-02,  4.9705e-01,  7.0042e-01,
         -3.4858e-01, -1.1527e+00,  3.6295e-01,  2.7313e-01,  2.9266e-01,
         -2.0905e-01,  1.2358e+00, -1.4073e-01,  1.3224e+00,  7.4463e-01,
          3.9619e-01, -1.9311e+00, -8.7972e-01,  2.2325e+00, -7.2284e-01,
          4.2627e-01,  9.2061e-01,  4.0705e-01, -1.6956e+00,  4.0305e-01,
         -5.7429e-01,  1.6643e-01, -2.6716e-02, -8.6688e-01, -1.1175e+00,
          8.3157e-01, -1.1545e-01,  3.3077e-01, -5.5019e-01,  2.0395e-01,
         -4.6982e-01,  5.2431e-01,  9.5747e-01, -1.0636e+00,  8.3819e-01,
          9.2096e-01,  8.5706e-01, -8.0152e-01,  1

In [2]:
import chromadb
import os

chroma_client = chromadb.PersistentClient(path='data/chroma2.db')
collection = chroma_client.get_or_create_collection(name='img', metadata={"hnsw:space": "cosine"})
img_root = 'data/train'

def embedding_func(img_root):
    imgs = os.listdir(img_root)
    for i, img in enumerate(imgs):
        img_path = os.path.join(img_root, img)
        print(img_path)
        id = os.path.splitext(img_path)[0]
        image = Image.open(img_path)
        inputs = feature_extractor(images=image, return_tensors="pt")
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
        collection.add(embeddings=embedding.tolist(), documents=[img_path], ids=[f"id{i}"])
embedding_func(img_root)




data/train\asparagus1_resized.jpeg
data/train\asparagus2_resized.jpg
data/train\asparagus3_resized.jpg
data/train\asparagus4_resized.jpeg
data/train\asparagus5_resized.jpg
data/train\asparagus6_resized.jpeg
data/train\burger1_resized.jpeg
data/train\burger2_resized.jpg
data/train\burger3_resized.jpeg
data/train\burger4_resized.jpeg
data/train\burger5_resized.jpg
data/train\burger6_resized.jpeg
data/train\fries1_resized.jpeg
data/train\fries2_resized.jpeg
data/train\fries3_resized.jpeg
data/train\fries4_resized.jpeg
data/train\fries5_resized.jpeg
data/train\fries6_resized.jpg
data/train\potato1_resized.jpg
data/train\potato2_resized.jpg
data/train\potato3_resized.jpg
data/train\potato4_resized.jpeg
data/train\potato5_resized.jpg
data/train\potato6_resized.jpeg
data/train\salad1_resized.jpeg
data/train\salad2_resized.jpeg
data/train\salad3_resized.jpeg
data/train\salad4_resized.jpg
data/train\salad5_resized.jpeg
data/train\salad6_resized.jpg
data/train\steak1_resized.jpg
data/train\steak

In [20]:
image = Image.open('data/test/burger_test.jpeg')
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
embeddings = outputs.last_hidden_state[:, 0, :]
res = collection.query(query_embeddings=embeddings.tolist(), n_results=5)
print(res)

{'ids': [['id9', 'id8', 'id11', 'id6', 'id10']], 'distances': [[0.3219032486751544, 0.3655042984959571, 0.3744061385984271, 0.3807297461676725, 0.38114302122953647]], 'metadatas': [[None, None, None, None, None]], 'embeddings': None, 'documents': [['data/train\\burger4_resized.jpeg', 'data/train\\burger3_resized.jpeg', 'data/train\\burger6_resized.jpeg', 'data/train\\burger1_resized.jpeg', 'data/train\\burger5_resized.jpg']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}
